# Laboratorio 3 - Clasificación multiclase

###En éste laboratorio se aplicacrá la regresión logística onevsall al dataset "Lab3_music_genre.csv" para hacer la predicción de distintos géneros de música. Se comenzará haciendo el preprocesamiento del dataset con Pandas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. Librerías

In [ ]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

#Libreria pandas y scikitlearn para el preprocesamiento de datasets
import pandas as pd
import sklearn as skl

#2. Contenido del dataset
En primer lugar, el contenido del dataset se carga en "dataframe", el cual Panda utilizará como unidad de información.\
Luego se muestra y describe con un resumen el contenido del dataframe.\
El dataset contiene un total de 32485 pistas de música con los que se puede predecir el género musical analizando varias características de las pistas musicales. Los datos contienen 13 características y la etiqueta y con 10 Clases (generos de música).

In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/SIS 420/Lab3/Lab3_music_genre.csv')
print(dataframe)
print(dataframe.describe())

       popularity  acousticness  danceability  duration_ms  energy  \
0            27.0       0.00468         0.652         -1.0   0.941   
1            31.0       0.01270         0.622     218293.0   0.890   
2            28.0       0.00306         0.620     215613.0   0.755   
3            34.0       0.02540         0.774     166875.0   0.700   
4            32.0       0.00465         0.638     222369.0   0.587   
...           ...           ...           ...          ...     ...   
50000        59.0       0.03340         0.913         -1.0   0.574   
50001        72.0       0.15700         0.709     251860.0   0.362   
50002        51.0       0.00597         0.693     189483.0   0.763   
50003        65.0       0.08310         0.782     262773.0   0.472   
50004        67.0       0.10200         0.862     267267.0   0.642   

       instrumentalness key  liveness  loudness   mode  speechiness    tempo  \
0               0.79200  A#     0.115    -5.201  Minor       0.0748  100.889   

#3.Preprocesamiento del dataset

###3.1. Preprocesamiento del dataset: Separar atributos(X) de la etiqueta (y) que se va a usar para el entrenamiento
Los atributos(X) se van a cargar en la variable "dataset" y la etiqueta "y" en la variable "label".

In [ ]:
dataset = dataframe.drop("music_genre", axis=1)
label = dataframe["music_genre"].copy()

###3.2. Preprocesamiento del dataset: Atributos con texto
En nuestro dataset tenemos 2 atributos NO NUMÉRICO que son "key" y "mode".

###3.2.1 CARACTERÍSTICA "key"

###Manejo de los datos Not a Number
Por un análisis minucioso de la columna "key" se ha verificado que existen valores faltantes o indefinidos en los datos, están vacías o no tienen un valor asignado (Not a Number). \
Limpieza por Imputación: Los valores faltantes se van a reemplazar con un valor sustituto. Para datos categóricos, una estrategia común es reemplazar los nan por la moda (el valor más frecuente) de la columna.\
Ésta imputación lo realizamos con herramientas de Pandas y scikit-learn:

In [ ]:
# Tenemos el dataFrame y 'key' es la columna con valores NaN
mode_value = dataframe['key'].mode()[0]  # Encuentra la moda de la columna
dataframe['key'].fillna(mode_value, inplace=True)  # Reemplaza NaN con la moda

###Ahora, los atributos de texto de la característica "key" lo cargamos en la variable key_cat y lo mostramos.

In [ ]:
key_cat = dataframe[['key']]
print(key_cat.head(12))

   key
0   A#
1    D
2   G#
3   C#
4   F#
5    D
6    B
7    G
8    F
9    A
10  F#
11   B


##Preprocesamiento con OneHotEncoder de sklearn
Ésto devuelve una codificación donde se indica el número de instancia dentro del dataset, número de atributo binario y el valor.
Con el método toarray() convertimos a una matriz y mostramos sus valores y tamaño (shape).

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
key_cat_one_hot = one_hot_encoder.fit_transform(key_cat)
print(key_cat_one_hot.toarray().shape)
print(key_cat_one_hot.toarray())

(50005, 12)
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


##Visualización de los nombres de todas las categorías de "key"

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Visualizar los nombres de las categorías
categories = one_hot_encoder.categories_

# Dado que key_cat es una sola columna, categories_ será una lista de un solo elemento.
# Accedemos a este primer elemento para obtener las categorías
print(categories[0])

['A' 'A#' 'B' 'C' 'C#' 'D' 'D#' 'E' 'F' 'F#' 'G' 'G#']


###Crear un DataFrame para las características codificadas, concatenar con las características originales y mostrar

In [ ]:
# Crear un DataFrame para las características codificadas
encoded_df = pd.DataFrame(key_cat_one_hot.toarray(), columns=['A','B','C','D','E','F','G','A#','C#','D#','F#','G#'])

# Concatenar las características originales con las características codificadas
df_nuevo = pd.concat([dataset, encoded_df], axis=1)

# Visualizar el nuevo DataFrame
print(df_nuevo.head())

   popularity  acousticness  danceability  duration_ms  energy  \
0        27.0       0.00468         0.652         -1.0   0.941   
1        31.0       0.01270         0.622     218293.0   0.890   
2        28.0       0.00306         0.620     215613.0   0.755   
3        34.0       0.02540         0.774     166875.0   0.700   
4        32.0       0.00465         0.638     222369.0   0.587   

   instrumentalness key  liveness  loudness   mode  ...    C    D    E    F  \
0           0.79200  A#     0.115    -5.201  Minor  ...  0.0  0.0  0.0  0.0   
1           0.95000   D     0.124    -7.043  Minor  ...  0.0  0.0  0.0  1.0   
2           0.01180  G#     0.534    -4.617  Major  ...  0.0  0.0  0.0  0.0   
3           0.00253  C#     0.157    -4.498  Major  ...  0.0  0.0  1.0  0.0   
4           0.90900  F#     0.157    -6.266  Major  ...  0.0  0.0  0.0  0.0   

     G   A#   C#   D#   F#   G#  
0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0

###3.2.2 CARACTERÍSTICA "mode"

###Manejo de los datos Not a Number por imputación

In [ ]:
# Tenemos el dataFrame y 'mode' es la columna con valores NaN
mode_value = dataframe['mode'].mode()[0]  # Encuentra la moda de la columna
dataframe['mode'].fillna(mode_value, inplace=True)  # Reemplaza NaN con la moda

###Los atributos de texto de la característica "mode" lo cargamos en la variable mode_cat y lo mostramos.

In [ ]:
mode_cat = dataframe[['mode']]
print(mode_cat.head(12))

     mode
0   Minor
1   Minor
2   Major
3   Major
4   Major
5   Minor
6   Major
7   Minor
8   Minor
9   Minor
10  Minor
11  Minor


##Preprocesamiento con OneHotEncoder de sklearn

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
mode_cat_one_hot = one_hot_encoder.fit_transform(mode_cat)
print(mode_cat_one_hot.toarray().shape)
print(mode_cat_one_hot.toarray())

(50005, 2)
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


##Visualización de los nombres de todas las categorías de "mode"

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Visualizar los nombres de las categorías
categories = one_hot_encoder.categories_

# Dado que key_cat es una sola columna, categories_ será una lista de un solo elemento.
# Accedemos a este primer elemento para obtener las categorías
print(categories[0])

['Major' 'Minor']


###Crear un DataFrame para las características codificadas, concatenar con las características originales y mostrar

In [ ]:
# Crear un DataFrame para las características codificadas
encoded_df = pd.DataFrame(mode_cat_one_hot.toarray(), columns=['Major','Minor'])

# Concatenar las características originales con las características codificadas
df_new = pd.concat([df_nuevo, encoded_df], axis=1)

# Visualizar el nuevo DataFrame
print(df_new.head())

   popularity  acousticness  danceability  duration_ms  energy  \
0        27.0       0.00468         0.652         -1.0   0.941   
1        31.0       0.01270         0.622     218293.0   0.890   
2        28.0       0.00306         0.620     215613.0   0.755   
3        34.0       0.02540         0.774     166875.0   0.700   
4        32.0       0.00465         0.638     222369.0   0.587   

   instrumentalness key  liveness  loudness   mode  ...    E    F    G   A#  \
0           0.79200  A#     0.115    -5.201  Minor  ...  0.0  0.0  0.0  0.0   
1           0.95000   D     0.124    -7.043  Minor  ...  0.0  1.0  0.0  0.0   
2           0.01180  G#     0.534    -4.617  Major  ...  0.0  0.0  0.0  0.0   
3           0.00253  C#     0.157    -4.498  Major  ...  1.0  0.0  0.0  0.0   
4           0.90900  F#     0.157    -6.266  Major  ...  0.0  0.0  0.0  0.0   

    C#   D#   F#   G#  Major  Minor  
0  0.0  0.0  0.0  0.0    0.0    1.0  
1  0.0  0.0  0.0  0.0    0.0    1.0  
2  0.0  0.0  0

###Eliminar las características originales "key" y "mode" del df_new

In [ ]:
# Eliminar la característica 'key' y 'mode' de df_new:
df_new_sin_key_sin_mode = df_new.drop(columns=['key', 'mode'])

# Visualizar el nuevo DataFrame sin las características 'key', 'mode'
print(df_new_sin_key_sin_mode.head())

   popularity  acousticness  danceability  duration_ms  energy  \
0        27.0       0.00468         0.652         -1.0   0.941   
1        31.0       0.01270         0.622     218293.0   0.890   
2        28.0       0.00306         0.620     215613.0   0.755   
3        34.0       0.02540         0.774     166875.0   0.700   
4        32.0       0.00465         0.638     222369.0   0.587   

   instrumentalness  liveness  loudness  speechiness    tempo  ...    E    F  \
0           0.79200     0.115    -5.201       0.0748  100.889  ...  0.0  0.0   
1           0.95000     0.124    -7.043       0.0300  115.002  ...  0.0  1.0   
2           0.01180     0.534    -4.617       0.0345  127.994  ...  0.0  0.0   
3           0.00253     0.157    -4.498       0.2390  128.014  ...  1.0  0.0   
4           0.90900     0.157    -6.266       0.0413  145.036  ...  0.0  0.0   

     G   A#   C#   D#   F#   G#  Major  Minor  
0  0.0  0.0  0.0  0.0  0.0  0.0    0.0    1.0  
1  0.0  0.0  0.0  0.0  0.0

###3.3. Preprocesamiento del dataset: Atributos duration_ms y tempo
En nuestro dataset tenemos 2 atributos numéricos donde hay datos indefinidos.

In [ ]:
import pandas as pd

# Primero, calculamos el promedio de los valores en 'duration_ms' excluyendo los valores -1.
mean_value = df_new_sin_key_sin_mode[df_new_sin_key_sin_mode['duration_ms'] != -1]['duration_ms'].mean()

# Luego, reemplazamos los valores -1 en 'duration_ms' con el valor promedio calculado.
df_new_sin_key_sin_mode['duration_ms'].replace(-1, mean_value, inplace=True)

In [ ]:
# Convierte los valores de 'tempo' a numéricos, forzando los errores a NaN (los '?' se convertirán en NaN).
df_new_sin_key_sin_mode['tempo'] = pd.to_numeric(df_new_sin_key_sin_mode['tempo'], errors='coerce')

# Calcula el promedio de 'tempo', excluyendo los NaN generados por los '?'.
mean_tempo = df_new_sin_key_sin_mode['tempo'].mean()

# Reemplaza los NaN en 'tempo' con el promedio calculado.
df_new_sin_key_sin_mode['tempo'].fillna(mean_tempo, inplace=True)

### Visualizar el dataset df_new_sin_key_sin_mode

In [ ]:
print(df_new_sin_key_sin_mode.head(10))

   popularity  acousticness  danceability    duration_ms  energy  \
0        27.0       0.00468         0.652  245503.541466   0.941   
1        31.0       0.01270         0.622  218293.000000   0.890   
2        28.0       0.00306         0.620  215613.000000   0.755   
3        34.0       0.02540         0.774  166875.000000   0.700   
4        32.0       0.00465         0.638  222369.000000   0.587   
5        47.0       0.00523         0.755  519468.000000   0.731   
6        46.0       0.02890         0.572  214408.000000   0.803   
7        43.0       0.02970         0.809  416132.000000   0.706   
8        39.0       0.00299         0.509  292800.000000   0.921   
9        22.0       0.00934         0.578  204800.000000   0.731   

   instrumentalness  liveness  loudness  speechiness       tempo  ...    E  \
0          0.792000    0.1150    -5.201       0.0748  100.889000  ...  0.0   
1          0.950000    0.1240    -7.043       0.0300  115.002000  ...  0.0   
2          0.0118

###3.4. Adecuar los valores de "music_genre"  ->  "y"
En los géneros de música tenemos 10 clases.\
Adecuamos a valores numéricos de esta forma:\
1 Electronic\
2 Anime\
3 Jazz\
4 Alternative\
5 Country\
6 Rap\
7 Blues\
8 Rock\
9 Classical\
10 Hip-Hop

In [ ]:
# Definir el mapeo de géneros musicales a números enteros.
genre_to_number = {
    'Electronic': 1,
    'Anime': 2,
    'Jazz': 3,
    'Alternative': 4,
    'Country': 5,
    'Rap': 6,
    'Blues': 7,
    'Rock': 8,
    'Classical': 9,
    'Hip-Hop': 10
}

# Aplicar el mapeo a la serie 'label'.
label_mapped = label.map(genre_to_number)

# Verificar los cambios realizando una impresión de los primeros elementos.
print(label_mapped.head())

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: music_genre, dtype: float64


###Concatenamos 'music_genre' a df_new_sin_key_sin_mode. El DataFrame final es: df_final

In [ ]:
# Previamente ya definimos df_new_sin_key_sin_mode y label_mapped
# Concatenamos 'music_genre' a df_new_sin_key_sin_mode de la siguiente manera:
df_final = pd.concat([df_new_sin_key_sin_mode, label_mapped], axis=1)

# Visualizar el nuevo DataFrame final
print(df_final.head())

   popularity  acousticness  danceability    duration_ms  energy  \
0        27.0       0.00468         0.652  245503.541466   0.941   
1        31.0       0.01270         0.622  218293.000000   0.890   
2        28.0       0.00306         0.620  215613.000000   0.755   
3        34.0       0.02540         0.774  166875.000000   0.700   
4        32.0       0.00465         0.638  222369.000000   0.587   

   instrumentalness  liveness  loudness  speechiness    tempo  ...    F    G  \
0           0.79200     0.115    -5.201       0.0748  100.889  ...  0.0  0.0   
1           0.95000     0.124    -7.043       0.0300  115.002  ...  1.0  0.0   
2           0.01180     0.534    -4.617       0.0345  127.994  ...  0.0  0.0   
3           0.00253     0.157    -4.498       0.2390  128.014  ...  0.0  0.0   
4           0.90900     0.157    -6.266       0.0413  145.036  ...  0.0  0.0   

    A#   C#   D#   F#   G#  Major  Minor  music_genre  
0  0.0  0.0  0.0  0.0  0.0    0.0    1.0          1.0 

###3.5. Limpieza de valores NaN en los atributos: popularity, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo. Asimismo en la etiqueta music_genre.
Se procede nuevamente a la limpieza de valores NaN para evitar que los valores calculados de all_theta sean "ceros".

In [ ]:
print(df_final.describe())

         popularity  acousticness  danceability   duration_ms        energy  \
count  50000.000000  50000.000000  50000.000000  5.000000e+04  50000.000000   
mean      44.220420      0.306383      0.558241  2.455035e+05      0.599755   
std       15.542008      0.341340      0.178632  1.057864e+05      0.264559   
min        0.000000      0.000000      0.059600  1.550900e+04      0.000792   
25%       34.000000      0.020000      0.442000  1.948750e+05      0.433000   
50%       45.000000      0.144000      0.568000  2.360270e+05      0.643000   
75%       56.000000      0.552000      0.687000  2.686122e+05      0.815000   
max       99.000000      0.996000      0.986000  4.830606e+06      0.999000   

       instrumentalness      liveness      loudness   speechiness  \
count      50000.000000  50000.000000  50000.000000  50000.000000   
mean           0.181601      0.193896     -9.133761      0.093586   
std            0.325409      0.161637      6.162990      0.101373   
min         

In [ ]:
# Lista de atributos con valores NaN
atributos_nan = ['popularity', 'acousticness', 'danceability', 'duration_ms',
                 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence',
                 'tempo', 'music_genre']

# Reemplazar NaN con el promedio de los valores no NaN para cada atributo
for atributo in atributos_nan:
    promedio = df_final[atributo].mean()  # Calcula el promedio
    df_final[atributo].fillna(promedio, inplace=True)  # Reemplaza NaN con el promedio

# Verificar que se hayan reemplazado correctamente
print(df_final.isnull().sum())

popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
A                   0
B                   0
C                   0
D                   0
E                   0
F                   0
G                   0
A#                  0
C#                  0
D#                  0
F#                  0
G#                  0
Major               0
Minor               0
music_genre         0
dtype: int64


In [ ]:
print(df_final.describe())

         popularity  acousticness  danceability   duration_ms        energy  \
count  50005.000000  50005.000000  50005.000000  5.000500e+04  50005.000000   
mean      44.220420      0.306383      0.558241  2.455035e+05      0.599755   
std       15.541231      0.341323      0.178623  1.057811e+05      0.264546   
min        0.000000      0.000000      0.059600  1.550900e+04      0.000792   
25%       34.000000      0.020100      0.442000  1.948750e+05      0.433000   
50%       45.000000      0.144000      0.568000  2.360400e+05      0.643000   
75%       56.000000      0.552000      0.687000  2.686000e+05      0.815000   
max       99.000000      0.996000      0.986000  4.830606e+06      0.999000   

       instrumentalness      liveness      loudness   speechiness  \
count      50005.000000  50005.000000  50005.000000  50005.000000   
mean           0.181601      0.193896     -9.133761      0.093586   
std            0.325393      0.161629      6.162682      0.101368   
min         

## 4. Aplicación de la regresión logística onevsall
###Al inicio cargamos el dataframe df_final en data.
###Hacemos también:  y[y == 10] = 0   para que argmax() identifique la última clase con 0.

In [ ]:
# La entrada es de 26 elemnetos contando el X0
input_layer_size  = 26

# 10 etiquetas, de 1 a 10 (tomar en cuenta que se asigna "0" a la etiqueta 10)
num_labels = 10

#  datos de entrenamiento almacenados en los arreglos X, y
data = df_final.copy()

# print(data['y'])
# print(data['y'].ravel())
# print(data['X'])
#X = data[:, :25]                                                #array de NumPy
#y = data[:, 25]

#Dataframe de Pandas
X = data.iloc[:, :25]  # Seleccionar todas las filas y las columnas de 0 a 24 (inclusive).
y = data.iloc[:, 25]   # Seleccionar todas las filas y solo la columna 25.

# X, y = data['X'], data['y'].ravel()
# print(X)
# print(y)
# establecer el dígito cero en 0, en lugar del 10 asignado a este conjunto de datos
# Esto se hace debido a que el conjunto de datos se utilizó en MATLAB donde no hay índice 0
y[y == 10] = 0
print(y)

m = y.size

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
50000    0.0
50001    0.0
50002    0.0
50003    0.0
50004    0.0
Name: music_genre, Length: 50005, dtype: float64


In [ ]:
#print(X[0,:])
#print(y)

#Dataframe de Pandas
print(X.iloc[0, :])  # Imprimir la primera fila de X
print(y)             # Imprimir la columna y completa

popularity              27.000000
acousticness             0.004680
danceability             0.652000
duration_ms         245503.541466
energy                   0.941000
instrumentalness         0.792000
liveness                 0.115000
loudness                -5.201000
speechiness              0.074800
tempo                  100.889000
valence                  0.759000
A                        0.000000
B                        1.000000
C                        0.000000
D                        0.000000
E                        0.000000
F                        0.000000
G                        0.000000
A#                       0.000000
C#                       0.000000
D#                       0.000000
F#                       0.000000
G#                       0.000000
Major                    0.000000
Minor                    1.000000
Name: 0, dtype: float64
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
50000    0.0
50001    0.0
50002    0.0
50003    

###Normalización

In [ ]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [ ]:
# llama featureNormalize con los datos cargados
X_norm, mu, sigma = featureNormalize(X)

In [ ]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
# Agraga el termino de intercepción a A
#X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)
X = X_norm
# X = np.concatenate([np.ones((m, 1)), X], axis=1)

### Visualización de los datos

En nuestro caso no podemos visualizar. Por ejemplo, se tendría que utilizar la biblioteca de visualización de datos Seaborn basada en matplotlib.


In [ ]:
def displayData(X, example_width=None, figsize=(10, 10)):
    """
    Muestra datos 2D almacenados en X en una cuadrícula apropiada.
    """
    # Calcula filas, columnas
    if X.ndim == 2:
        m, n = X.shape
    elif X.ndim == 1:
        n = X.size
        m = 1
        X = X[None]  # Promocionar a una matriz bidimensional
    else:
        raise IndexError('La entrada X debe ser 1 o 2 dimensinal.')

    example_width = example_width or int(np.round(np.sqrt(n)))
    example_height = n / example_width

    # Calcula el numero de elementos a mostrar
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, ax_array = pyplot.subplots(display_rows, display_cols, figsize=figsize)
    fig.subplots_adjust(wspace=0.025, hspace=0.025)

    ax_array = [ax_array] if m == 1 else ax_array.ravel()

    for i, ax in enumerate(ax_array):
        ax.imshow(X[i].reshape(example_width, example_width, order='F'),
                  cmap='Greys', extent=[0, 1, 0, 1])
        ax.axis('off')

In [ ]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
#rand_indices = np.random.choice(m, 100, replace=False)
#sel = X[rand_indices, :]

# displayData(sel)

### Vectorización de la función de costo

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    """
    Calcula el costo de usar theta como parámetro para la regresión logística regularizada y
    el gradiente del costo w.r.t. a los parámetros.

    Parametros
    ----------
    theta : array_like
        Parametro theta de la regresion logistica. Vector de la forma(shape) (n, ). n es el numero de caracteristicas
        incluida la intercepcion

    X : array_like
        Dataset con la forma(shape) (m x n). m es el numero de ejemplos, y n es el numero de
        caracteristicas (incluida la intercepcion).

    y : array_like
        El conjunto de etiquetas. Un vector con la forma (shape) (m, ). m es el numero de ejemplos

    lambda_ : float
        Parametro de regularización.

    Devuelve
    -------
    J : float
        El valor calculado para la funcion de costo regularizada.

    grad : array_like
        Un vector de la forma (shape) (n, ) que es el gradiente de la
        función de costo con respecto a theta, en los valores actuales de theta..
    """
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

### Clasificacion One-vs-all   (con vectorización regularizada)


In [ ]:
from sklearn.model_selection import train_test_split

def oneVsAll(X, y, num_labels, lambda_):
    """
    Trains num_labels logistic regression classifiers and returns
    each of these classifiers in a matrix all_theta, where the i-th
    row of all_theta corresponds to the classifier for label i.

    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). m is the number of
        data points, and n is the number of features. Note that we
        do not assume that the intercept term (or bias) is in X, however
        we provide the code below to add the bias term to X.

    y : array_like
        The data labels. A vector of shape (m, ).

    num_labels : int
        Number of possible labels.

    lambda_ : float
        The logistic regularization parameter.

    Returns
    -------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        (ie. `numlabels`) and n is number of features without the bias.
    """
    # algunas variables utiles
    #m, n = X.shape

    #all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    #X = np.concatenate([np.ones((m, 1)), X], axis=1)

    #for c in np.arange(num_labels):
        #initial_theta = np.zeros(n + 1)
        #options = {'maxiter': 100}                             #MÁXIMO NUMERO DE ITERACIONES
        #res = optimize.minimize(lrCostFunction,
                                #initial_theta,
                                #(X, (y == c), lambda_),
                                #jac=True,
                                #method='CG',
                                #options=options)

        #all_theta[c] = res.x

    #return all_theta

    # DIVIDIR LOS DATOS EN CONJUNTO DE ENTRENAMIENTO Y PRUEBA (80% entrenamiento, 20% prueba)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Algunas variables útiles
    m, n = X_train.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agregar unos a la matriz X de entrenamiento
    X_train = np.concatenate([np.ones((m, 1)), X_train], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 100}                             # MÁXIMO NUMERO DE ITERACIONES
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X_train, (y_train == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [ ]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 26)


In [ ]:
print(all_theta)

[[-3.81145216e+00  9.12460000e-01 -1.13529517e-01  9.32283523e-01
  -5.39401668e-02 -1.32653420e-01 -1.03118584e+00  1.02207717e-01
   3.47845692e-01  5.29048244e-01 -2.12943819e-02 -2.81909418e-01
  -6.97447886e-03  1.09607636e-02 -1.58651108e-02  9.06524315e-03
   7.79775512e-02 -4.57287377e-02 -5.19487369e-02 -3.19244465e-02
   9.26216669e-03  1.93180185e-02 -3.52854726e-02  4.82264302e-02
  -8.08508120e-02  8.08508120e-02]
 [-3.16531430e+00 -6.98720620e-01 -6.11100099e-01  1.06903489e+00
   2.51388774e-01  9.27226188e-01  7.50064025e-01  5.35321362e-02
  -2.69720018e-03 -3.64563326e-02  1.90218899e-01 -8.57777949e-01
  -1.42258455e-03  6.72691645e-03  1.96818402e-02 -4.31659971e-03
   4.50911554e-02 -4.92581113e-02 -2.03399160e-02 -5.76219672e-02
   5.82735958e-03  4.30227608e-02  6.44183184e-04  1.02075582e-02
  -9.01953453e-02  9.01953453e-02]
 [-3.93102729e+00 -2.11496301e+00  9.92713730e-02 -5.14982890e-01
  -3.87636620e-01  1.99319114e-01  2.13781849e-01 -2.33133692e-01
   1.2


#### Prediccion One-vs-all

In [ ]:
def predictOneVsAll(all_theta, X):
    """
    Devuelve un vector de predicciones para cada ejemplo en la matriz X.
    Tenga en cuenta que X contiene los ejemplos en filas.
    all_theta es una matriz donde la i-ésima fila es un vector theta de regresión logística entrenada para la i-ésima clase.
    Debe establecer p en un vector de valores de 0..K-1 (por ejemplo, p = [0, 2, 0, 1]
    predice clases 0, 2, 0, 1 para 4 ejemplos).

    Parametros
    ----------
    all_theta : array_like
        The trained parameters for logistic regression for each class.
        This is a matrix of shape (K x n+1) where K is number of classes
        and n is number of features without the bias.

    X : array_like
        Data points to predict their labels. This is a matrix of shape
        (m x n) where m is number of data points to predict, and n is number
        of features without the bias term. Note we add the bias term for X in
        this function.

    Devuelve
    -------
    p : array_like
        The predictions for each data point in X. This is a vector of shape (m, ).
    """

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

###Precisión del conjunto de entrenamiento

In [ ]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
#XPrueba = X[10:150, :].copy()
#Pandas
XPrueba = X.iloc[40001:50004, :].copy()
print(XPrueba.shape)
#print(np.ones((1)))
#print(XPrueba)
#p = np.zeros(1)
XPrueba = np.concatenate([np.ones((10003, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)    #es la prediccion

# displayData(X[1002:1003, :])
print(y[40001:50004])                                                                  #SON LOS VALORES VERDADEROS

(50005, 25)
Precision del conjuto de entrenamiento: 51.05%
(10003, 25)
(10003, 26)
[1 8 8 ... 6 6 6]
40001    8.0
40002    8.0
40003    8.0
40004    8.0
40005    9.0
        ... 
49999    0.0
50000    0.0
50001    0.0
50002    0.0
50003    0.0
Name: music_genre, Length: 10003, dtype: float64
